In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!apt-get update # to update ubuntu to correctly run apt install

In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import OrderedDict
from __future__ import unicode_literals
import re
import time

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


## Extract the job title and links

In [ ]:
# specify the strings for URL
# url1 is the first part
# url2 has different keywords

url1 = 'https://www.amazon.jobs/en/search?offset='
url2 = '&result_limit=10&sort=relevant&country%5B%5D=USA&country%5B%5D=IND&country%5B%5D=GBR&country%5B%5D=CAN&country%5B%5D=CRI&country%5B%5D=AUS&distanceType=Mi&radius=24km&latitude=&longitude=&loc_group_id=&loc_query=&base_query=&city=&country=&region=&county=&query_options=&'

In [ ]:
def amazon_get_jobs(url1, url2):
    """
    retrieve job titles and links for each page
    """
    job_title=[]
    job_link=[]
    page_num = 0

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    URL = url1 + str(page_num*10) + url2
    driver.get(URL)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    while soup.find_all("h3", {"class": "job-title"}):                 
        job_title.extend([td.text for td in soup.findAll("h3", {"class": "job-title"})])
        job_link.extend(['https://www.amazon.jobs'+td.find('a').get('href') for td in soup.findAll("div", {"class": "job-tile"})])
        driver.quit()

        page_num += 1        
        driver=webdriver.Chrome('chromedriver',options=chrome_options)
        URL = url1 + str(page_num*10) + url2
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()   
    return job_title,job_link

In [ ]:
# get job titles and links for different categories of jobs
job_title, job_link = amazon_get_jobs(url1, url2)

In [ ]:
print(len(job_title))

2030


In [ ]:
job_link[0:5]

['https://www.amazon.jobs/en/jobs/2293554/sap-data-governance-sap-mdg-architect',
 'https://www.amazon.jobs/en/jobs/2318733/senior-finance-analyst-eu-amzl',
 'https://www.amazon.jobs/en/jobs/2318992/business-analytics-leader-aws-sales-insights-analytics-and-data-science-siads',
 'https://www.amazon.jobs/en/jobs/2318993/partner-sales-business-analyst-amer-partner-sales-operations',
 'https://www.amazon.jobs/en/jobs/2318113/accounts-receivable-sr-financial-analyst-nam-aws-nam']

In [ ]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

print(len(df_title_link))

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()
print(len(df_title_link))

2030
2030


In [ ]:
# save the titles and links to csv
df_title_link.to_csv('./drive/MyDrive/Swatcloud/web_scraper/amazon_jobs_titles_links_all.csv')

## Extract job qualifications and save them to a dataframe

In [ ]:
# load in the job titles and links from the csv file
df_title_link = pd.read_csv('./drive/MyDrive/Swatcloud/web_scraper/amazon_jobs_titles_links_all.csv', index_col=0)
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,SAP Data Governance (SAP MDG) Architect,https://www.amazon.jobs/en/jobs/2293554/sap-da...
1,Senior Finance Analyst EU AMZL,https://www.amazon.jobs/en/jobs/2318733/senior...
2,"Business Analytics Leader, AWS, Sales Insights...",https://www.amazon.jobs/en/jobs/2318992/busine...
3,"Partner Sales Business Analyst, AMER Partner S...",https://www.amazon.jobs/en/jobs/2318993/partne...
4,Accounts Receivable Sr. Financial Analyst - NA...,https://www.amazon.jobs/en/jobs/2318113/accoun...


In [ ]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        time.sleep(2)
        URL=link[i]
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        s = ''
        d = ''
        try:
          tag = soup.find("h2", text='BASIC QUALIFICATIONS').parent
          if tag.find('li'):
            for t in tag.find_all('li'):
              s = s + ' ' + t.text
          else:
            s = s + ' ' + tag.text
        except:
          print(i, 'no mini qual')

        try:
          tag = soup.find("h2", text='PREFERRED QUALIFICATIONS').parent
          if tag.find('li'):
            for t in tag.find_all('li'):
              s = s + ' ' + t.text
          else:
            s = s + ' ' + tag.text
        except:
          print(i, 'no pref qual')

        try:
          tag = soup.find("h2", text='DESCRIPTION').parent
          if tag:
            d = tag.text
        except:
          print(i, 'no description')
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])

 
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [ ]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

77 no mini qual
77 no pref qual
77 no description
116 no pref qual
117 no pref qual
118 no pref qual
130 no pref qual
168 no pref qual
169 no pref qual
170 no pref qual
171 no pref qual
244 no pref qual
284 no pref qual
297 no mini qual
297 no pref qual
297 no description
298 no mini qual
298 no pref qual
298 no description
302 no mini qual
302 no pref qual
302 no description
311 no mini qual
311 no pref qual
311 no description
372 no mini qual
372 no pref qual
372 no description
394 no pref qual
404 no mini qual
404 no pref qual
404 no description
407 no pref qual
463 no mini qual
463 no pref qual
463 no description
494 no pref qual
512 no mini qual
512 no pref qual
512 no description
516 no mini qual
516 no pref qual
516 no description
557 no pref qual
623 no pref qual
633 no pref qual
634 no mini qual
634 no pref qual
634 no description
687 no pref qual
689 no mini qual
689 no pref qual
689 no description
732 no mini qual
732 no pref qual
732 no description
778 no pref qual
843 no m

In [ ]:
def remove_strings(s):
  s_new = s.replace('BASIC QUALIFICATIONS', ' ').replace('PREFERRED QUALIFICATIONS', ' ').replace('DESCRIPTION', ' ')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [ ]:
qual_cleaned = [remove_strings(q) for q in qual]
descrp_cleaned = [remove_strings(q) for q in descrp]

In [ ]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Amazon'
df = df.iloc[:, [4, 0, 1, 2, 3]]
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Amazon,SAP Data Governance (SAP MDG) Architect,• +5 years hands-on end to end implementation...,https://www.amazon.jobs/en/jobs/2293554/sap-da...,Have you ever wanted to be part of a team who...
1,Amazon,Senior Finance Analyst EU AMZL,· Experience in finance and and Degree in Fin...,https://www.amazon.jobs/en/jobs/2318733/senior...,Have you ever ordered a product on Amazon and...
2,Amazon,"Business Analytics Leader, AWS, Sales Insights...",Bachelor’s degree or higher in an analytical ...,https://www.amazon.jobs/en/jobs/2318992/busine...,The AWS Worldwide Commercial Sales Operations...
3,Amazon,"Partner Sales Business Analyst, AMER Partner S...",Bachelor's degree or equivalent experience 2+...,https://www.amazon.jobs/en/jobs/2318993/partne...,"For more than 11 years, Amazon Web Services (..."
4,Amazon,Accounts Receivable Sr. Financial Analyst - NA...,5+ years of post-qualification experience in ...,https://www.amazon.jobs/en/jobs/2318113/accoun...,Amazon's Finance Operations department seeks ...


In [ ]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=15 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
len(empty_mini_q)

47

In [ ]:
# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)
df.shape

(1983, 5)

In [ ]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
df_nodup.shape

(1983, 5)

In [ ]:
df_nodup.to_csv('./drive/MyDrive/Swatcloud/cleaned_data/individual_companies/amazon_all_jobs_cleaned.csv')